In [1]:
!pip install -q transformers accelerate datasets scikit-learn pandas

import os
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup,
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# For reproducibility
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)


Using device: cuda


In [2]:

df = pd.read_csv("final_10k_dataset_cleaned.csv")
print("Loaded dataset shape:", df.shape)
df.head()


Loaded dataset shape: (9996, 8)


,text,final_category,coarse_label,human_label_original,llm_pred_category,llm_raw_response,label_source,dataset_source
0,@Aiiane @aquarianfool it's important to let pe...,cyberbullying,toxic,other_cyberbullying,NaN,NaN,human_direct,cyberbullying_kaggle
1,I still remembered how much i bullied him. Nye...,cyberbullying,toxic,other_cyberbullying,NaN,NaN,human_direct,cyberbullying_kaggle
2,"@azmoderate @JoeWSJ Again, you talk like an id...",cyberbullying,toxic,religion,NaN,NaN,human_direct,cyberbullying_kaggle
3,Losing lives is much more important. Maybe was...,cyberbullying,toxic,age,NaN,NaN,human_direct,cyberbullying_kaggle
4,I taught anti-bullying units to my 7th graders...,cyberbullying,toxic,age,NaN,NaN,human_direct,cyberbullying_kaggle


In [3]:
# Check available columns
print(df.columns)

# Ensure required columns exist
assert "text" in df.columns
assert "final_category" in df.columns
assert "dataset_source" in df.columns

# Build label mapping from final_category to ids
unique_labels = sorted(df["final_category"].unique())
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

num_labels = len(label2id)
print("Labels:", label2id)
print("Number of labels:", num_labels)

# Map labels to ids
df["label_id"] = df["final_category"].map(label2id)

# See available dataset sources
print("\nAvailable dataset_source values:")
print(df["dataset_source"].value_counts())

# ==== CONFIGURABLE: choose one dataset_source and one model ====
DATASET_SOURCE = "jigsaw_toxic"        # change to "jigsaw_bias" or "cyberbullying_kaggle"
MODEL_NAME = "microsoft/deberta-v3-base"  # you can also try "roberta-base", "bert-base-uncased"

print(f"\nUsing subset where dataset_source == '{DATASET_SOURCE}'")
df_sub = df[df["dataset_source"] == DATASET_SOURCE].reset_index(drop=True)
print("Subset shape:", df_sub.shape)
print(df_sub["final_category"].value_counts())


Index(['text', 'final_category', 'coarse_label', 'human_label_original',
       'llm_pred_category', 'llm_raw_response', 'label_source',
       'dataset_source'],
      dtype='object')
Labels: {'abusive_violence': 0, 'cyberbullying': 1, 'generic_neutral': 2, 'hate_speech': 3, 'motivational_non_hate': 4, 'non_abusive_violence': 5, 'non_cyberbullying': 6, 'non_sexual': 7, 'other_toxic': 8, 'sexual_harassment': 9}
Number of labels: 10

Available dataset_source values:
dataset_source
jigsaw_bias             5713
jigsaw_toxic            2285
cyberbullying_kaggle    1998
Name: count, dtype: int64

Using subset where dataset_source == 'jigsaw_toxic'
Subset shape: (2285, 9)
final_category
generic_neutral      1000
other_toxic           558
hate_speech           391
abusive_violence      221
sexual_harassment     115
Name: count, dtype: int64


In [4]:
train_df, val_df = train_test_split(
    df_sub,
    test_size=0.2,
    random_state=42,
    stratify=df_sub["label_id"],
)

print("Train shape:", train_df.shape)
print("Val shape:", val_df.shape)


Train shape: (1828, 9)
Val shape: (457, 9)


In [5]:
class ToxicDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = list(texts)
        self.labels = list(labels)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        item = {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        }
        return item

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Build datasets
train_dataset = ToxicDataset(
    texts=train_df["text"].tolist(),
    labels=train_df["label_id"].tolist(),
    tokenizer=tokenizer,
    max_length=256,
)

val_dataset = ToxicDataset(
    texts=val_df["text"].tolist(),
    labels=val_df["label_id"].tolist(),
    tokenizer=tokenizer,
    max_length=256,
)

# DataLoaders
BATCH_SIZE = 16

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Train batches: 115
Val batches: 29


In [10]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

model.to(device)

EPOCHS = 5
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Total steps
total_steps = len(train_loader) * EPOCHS
warmup_steps = int(0.1 * total_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps,
)

print("Total training steps:", total_steps)


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total training steps: 575


In [11]:
def run_epoch(model, data_loader, optimizer=None, scheduler=None, train=True):
    if train:
        model.train()
    else:
        model.eval()

    all_labels = []
    all_preds = []
    total_loss = 0.0

    loop = tqdm(data_loader, desc="Train" if train else "Val")

    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.set_grad_enabled(train):
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            loss = outputs.loss
            logits = outputs.logits

            if train:
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                if scheduler is not None:
                    scheduler.step()

        total_loss += loss.item() * input_ids.size(0)

        preds = torch.argmax(logits, dim=-1)

        all_labels.extend(labels.detach().cpu().numpy().tolist())
        all_preds.extend(preds.detach().cpu().numpy().tolist())

    avg_loss = total_loss / len(data_loader.dataset)
    acc = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average="macro")

    return avg_loss, acc, macro_f1


for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    train_loss, train_acc, train_f1 = run_epoch(
        model, train_loader, optimizer=optimizer, scheduler=scheduler, train=True
    )
    print(
        f"Train   - Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Macro F1: {train_f1:.4f}"
    )

    val_loss, val_acc, val_f1 = run_epoch(
        model, val_loader, optimizer=None, scheduler=None, train=False
    )
    print(
        f"Val     - Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | Macro F1: {val_f1:.4f}"
    )



Epoch 1/5


Train:   0%|          | 0/115 [00:00<?, ?it/s]

Train   - Loss: 1.5483 | Acc: 0.4573 | Macro F1: 0.1563


Val:   0%|          | 0/29 [00:00<?, ?it/s]

Val     - Loss: 0.8579 | Acc: 0.6871 | Macro F1: 0.4175

Epoch 2/5


...

Val     - Loss: 0.7107 | Acc: 0.7484 | Macro F1: 0.5893

Epoch 5/5


Train:   0%|          | 0/115 [00:00<?, ?it/s]

Train   - Loss: 0.4011 | Acc: 0.8621 | Macro F1: 0.7242


Val:   0%|          | 0/29 [00:00<?, ?it/s]

Val     - Loss: 0.7410 | Acc: 0.7199 | Macro F1: 0.5712


In [12]:
output_dir = f"model_exp1_{DATASET_SOURCE.replace('/', '_')}_{MODEL_NAME.split('/')[-1]}"
os.makedirs(output_dir, exist_ok=True)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Save label2id mapping
import json
with open(os.path.join(output_dir, "label2id.json"), "w") as f:
    json.dump(label2id, f, indent=2)

print("Model and tokenizer saved to:", output_dir)


Model and tokenizer saved to: model_exp1_jigsaw_toxic_deberta-v3-base
